# Helix

In [1]:
from polymesh.recipes import circular_helix
fnc = circular_helix(1, 5)
fnc(1)

(0.5403023058681398, 0.8414709848078965, 5)

In [2]:
from polymesh.space import CartesianFrame
from polymesh import PolyData, LineData
import numpy as np

frame = CartesianFrame(dim=3)

space = np.linspace(0, 20*np.pi, 300)
coords = np.array(list(map(circular_helix(8, 2), space)))
topo = np.zeros((coords.shape[0]-1, 2))
topo[:, 0] = np.arange(topo.shape[0])
topo[:, 1] = topo[:, 0] + 1
topo = topo.astype(int)

mesh = PolyData(coords=coords, frame=frame)
mesh['helix'] = LineData(topo=topo, frame=frame)

mesh.plot(notebook=False)

In [3]:
coords[0], coords[-1]

(array([8., 0., 0.]),
 array([ 8.00000000e+00, -1.95943488e-14,  1.25663706e+02]))

In [4]:
from sigmaepsilon import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud
from polymesh.utils.space import frames_of_lines
from sigmaepsilon.fem.cells import B2 as Beam

import numpy as np
from numpy import pi as PI

w, h = 10., 10.  # width and height of the rectangular cross section
F = 10000.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = w * h**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[-1, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution(store='dofsol')
forces = structure.nodal_forces()
reactions = structure.reaction_forces()


In [5]:
dofsol[-1], forces[-1], reactions[0]

(array([-1.69533299e+01,  7.70038552e-01,  4.57891296e+00, -1.05684463e-10,
        -2.69820626e-01,  1.66253126e-02]),
 array([    0.,     0., 10000.,     0.,     0.,     0.]),
 array([-2.66161003e-06, -1.50777678e-07, -1.00000000e+04,  3.10896019e-05,
        -3.76386212e-04, -1.35950297e-05]))

In [6]:
forces[-1, 2], reactions[0, 2]

(10000.0, -10000.000005007245)